In [1]:
from blockcypher import get_address_full
from blockcypher import get_transaction_details

from tinyec.ec import SubGroup, Curve
import numpy as np
import hashlib
import libnum
import bech32
import binascii

from cryptos import *

import ecdsa
from ecdsa import SigningKey, SECP256k1
import hashlib
import base58
from ecdsa.util import sigencode_der, sigdecode_der

from embit.util import secp256k1
from embit.util import py_secp256k1

In [2]:
!pip install embit 

  Using cached embit-0.7.0-py3-none-any.whl


# SCRIPT TO FETCH ALL INFO FROM BLOCKCYPHER API INSTEAD  OF HARDCODING THE PARAMETERS

In [2]:
txn_details = get_transaction_details('8801efe18da3d4f199a4a4e12450a66c60a60c208f7822701bbc7defb5ef0b0a', include_hex=True)

In [3]:
txn_details

{'block_hash': '00000000000000000001670a0549a38ecf2e51d40936c3cfdc2da4804e7b6e32',
 'block_height': 826109,
 'block_index': 3931,
 'hash': '8801efe18da3d4f199a4a4e12450a66c60a60c208f7822701bbc7defb5ef0b0a',
 'hex': '02000000000101524e3e436f25df204ce27899af6ad638b44151bac5c3cf8a6a52116d1d659d8b0000000000ffffffff02e803000000000000160014ffa40104eec28c18911c803f1207f8fb6d993fc819350000000000001600145d50044454d3d18006e71b0da30d5c39b793053f0247304402201626d2d8533e6f3f9150a88e565068037e012ba156ebd376666f05a9da1fb7e702206b37258e900391bc1f104ee4556830730a7d0291c5fb8f52eb15bcbbe9edbd62012103fb561eb15488bc679156766a55b7e6c5a382d6efbfe80cbe6c65ab05e75bc96600000000',
 'addresses': ['bc1q8z7vfvl5jk9z0vdckphf6wsdjfce6qhgskp84e',
  'bc1ql7jqzp8wc2xp3ygusql3yplcldkej07gk7ga9n',
  'bc1qt4gqg3z560gcqph8rvx6xr2u8xmexpfl0mt7mp'],
 'total': 14593,
 'fees': 6716,
 'size': 222,
 'vsize': 141,
 'preference': 'low',
 'relayed_by': '88.198.5.130:8333',
 'confirmed': datetime.datetime(2024, 1, 17, 9, 13, 38, tzin

In [2]:
def double_sha256(data):
    
    in_bytes = bytes.fromhex(data)
    
    # First SHA256 hash
    first_hash = hashlib.sha256(in_bytes).digest()

    # Second SHA256 hash
    second_hash = hashlib.sha256(first_hash).digest()

    return second_hash.hex()

# Extracting signature from witness

In [5]:
input_num = int(txn_details['hex'][12:14], 16)
print(input_num)

1


In [6]:
sig = txn_details["inputs"][0]["witness"][0]
sig_hex = hex(int(sig, 16))
print(sig_hex)

0x304402201626d2d8533e6f3f9150a88e565068037e012ba156ebd376666f05a9da1fb7e702206b37258e900391bc1f104ee4556830730a7d0291c5fb8f52eb15bcbbe9edbd6201


# Extracting 'r' and 's' from signature

In [109]:
r_len = 2*int(sig_hex[8:10], 16)
r_end_idx = (10 + r_len)
r_ = sig_hex[10:r_end_idx]


s_len = 2*int(sig_hex[(r_end_idx + 2):(r_end_idx + 2 + 2)], 16)
s_end_idx = (r_end_idx + 2 + 2 + s_len)
s_ = sig_hex[(r_end_idx + 2 + 2):s_end_idx]

print(int(r_len/2))
print(r_)

print(int(s_len/2))
print(s_)

32
1626d2d8533e6f3f9150a88e565068037e012ba156ebd376666f05a9da1fb7e7
32
6b37258e900391bc1f104ee4556830730a7d0291c5fb8f52eb15bcbbe9edbd62


# Creating segwit scriptPubKey 

In [3]:
def calc_scriptcode(addr):# calculate segwit input address scriptpubkey 

#address = txn_details["inputs"][0]["addresses"][0]
    #address = 'bc1qrkyer6900d99c44jmdrh6f4g3dkjk90zf6ntuy' 
#address = 'tb1qt9xzu0df95vsfal8eptzyruv4e00k4ty6d8zhh'
    hrp = addr[0:2]

    address_decoded = bech32.decode(hrp, addr)
#address_decoded = segwit_addr.bech32_decode(address)

#print(address_decoded[2])
#print(address_decoded[2])

    byte_string = bytes(address_decoded[1])
    hex_string = binascii.hexlify(byte_string).decode('utf-8')
    scriptpubkey = '19' + '76' + 'a9' + '14' + hex_string + '88' + 'ac' 
    
    #print(scriptpubkey)
    
    return scriptpubkey
    

In [8]:
calc_scriptcode('bc1qfmsav54qgj8af49flwaymla7a56nv5gszss2u7')

'1976a9144ee1d652a0448fd4d4a9fbba4dffbeed3536511088ac'

In [10]:
addr = 'bc1qrkyer6900d99c44jmdrh6f4g3dkjk90zf6ntuy'
sc = calc_scriptcode(addr)
print(sc)

1976a9141d8991e8af7b4a5c56b2db477d26a88b6d2b15e288ac


In [11]:
# a different version, initial v1
# this is for generating script for output

#hex_string = binascii.hexlify(byte_string).decode('utf-8')

#scriptpubkey = '00' + '14' + hex_string 
#print("address : ", address)
#print("scriptpubkey : ", scriptpubkey)

In [12]:
# code to decode scriptpubkey to address 
# can use to cross check 
# this is for output not input 

spk = binascii.unhexlify('0014d2caa7b08db89cd62c9af34da53332d30e53bb15')
version = spk[0] - 0x50 if spk[0] else 0
program = spk[2:]
print(bech32.encode('bc', version, program))

bc1q6t920vydhzwdvty67dx62vej6v898wc4ly892a


# Creating unsigned transaction

## Things we need 
### 1. Version 
### 2. Previous transaction ID and the output index (to hash)
### 3. Sequence (to hash)
### 4. Outpoint

In [13]:
ver = txn_details['hex'][0:8]
print(ver)

02000000


In [14]:
prev_out_start_idx = 14
prev_out_end_idx = prev_out_start_idx + 72 

seq_start_idx = prev_out_end_idx + 2
seq_end_idx = seq_start_idx + 8 

prev_outs_list = []
seq_list = []

for i in range(input_num):
    
    # fetching prev. outs and this list will contain outpoints of all segwit inputs
    prev_outs_list.append(txn_details['hex'][prev_out_start_idx:prev_out_end_idx]) 
    
    # fetching sequence
    seq_list.append(txn_details['hex'][seq_start_idx:seq_end_idx])
    
    # update indecies
    prev_out_start_idx = prev_out_start_idx + 10
    prev_out_end_idx = prev_out_start_idx + 72
    
    seq_start_idx = prev_out_end_idx + 2
    seq_end_idx = seq_start_idx + 8    
    
prev_outs  = ' '.join(prev_outs_list)
seq = ' '.join(seq_list)

print(prev_outs)
print(seq)
print(seq_list)
print(prev_outs_list)

524e3e436f25df204ce27899af6ad638b44151bac5c3cf8a6a52116d1d659d8b00000000
ffffffff
['ffffffff']
['524e3e436f25df204ce27899af6ad638b44151bac5c3cf8a6a52116d1d659d8b00000000']


In [15]:
prev_outs_hash = double_sha256(prev_outs)
print(prev_outs_hash)

seq_hash = double_sha256(seq)
print(seq_hash)

06f2b86854f8d3e7d86cb1e39c2b09919adde421fed7a141e6515ffd7f88a684
3bb13029ce7b1f559ef5e747fcac439f1455a2ec7c5f09b72290795e70665044



### 5. scriptPubkey/scriptCode
### 6. Amount
### 7. Sequence
### 8. hashOutputs (amount || scriptPubKey size || scriptPubKey)
### 9. LockTime
### 10. HashType

In [16]:
addr = address = txn_details["inputs"][0]["addresses"][0]

In [11]:
calc_scriptcode('bc1qfmsav54qgj8af49flwaymla7a56nv5gszss2u7')


'1976a9144ee1d652a0448fd4d4a9fbba4dffbeed3536511088ac'

In [10]:
calc_amount(13340)

'1c34000000000000'

In [9]:
def calc_amount(val):
    amount_sats = val
    amount_sats_hex = hex(amount_sats)[2:] 


    if len(amount_sats_hex) % 2 != 0:
        amount_sats_hex = '0' + amount_sats_hex

    # Corrected line: remove [2:] before using fromhex()
    little_endian_bytes = bytes.fromhex(amount_sats_hex)[::-1]
    little_endian_padded = little_endian_bytes.ljust(8, b'\x00')


    # Print the result
    amount = little_endian_padded.hex()
    
    return amount



In [21]:
print(calc_amount(13340))
#print('00000080')

print(hex(501))


1c34000000000000
0x1f5


In [22]:
output_amount = []
output_scriptcode = []
to_hash_list = []

for i in range(len(txn_details["outputs"])):
    amount = calc_amount(txn_details["outputs"][i]['value'])
    output_amount.append(amount)
    
    scriptcode = calc_scriptcode(txn_details["outputs"][i]["addresses"][0])
    output_scriptcode.append(scriptcode)
                                 
    to_hash_list.append(amount)
    to_hash_list.append(scriptcode)
    

print(to_hash_list)
to_hash = ''.join(to_hash_list)

print(to_hash)
to_hashoutputs = to_hash

hashoutputs = sha256(to_hashoutputs)

print("To hashoutput : ", to_hashoutputs)
print("hashoutputs  : ", hashoutputs)

['e803000000000000', '1976a914ffa40104eec28c18911c803f1207f8fb6d993fc888ac', '1935000000000000', '1976a9145d50044454d3d18006e71b0da30d5c39b793053f88ac']
e8030000000000001976a914ffa40104eec28c18911c803f1207f8fb6d993fc888ac19350000000000001976a9145d50044454d3d18006e71b0da30d5c39b793053f88ac
To hashoutput :  e8030000000000001976a914ffa40104eec28c18911c803f1207f8fb6d993fc888ac19350000000000001976a9145d50044454d3d18006e71b0da30d5c39b793053f88ac
hashoutputs  :  dad860a5eede12e4b1e2e5596a2f0d8bfde2afa0448626ab562efde53d98507e


In [23]:
locktime = txn_details['hex'][-8:]
sighash = '01000000'

# Serializing and hashing

In [27]:
unsigned_txn_1 = ver + prev_outs_hash  + seq_hash  + prev_outs_list[0]  + scriptcode  + amount  + seq  + hashoutputs  + locktime + sighash

In [28]:
print(unsigned_txn_1)

0200000006f2b86854f8d3e7d86cb1e39c2b09919adde421fed7a141e6515ffd7f88a6843bb13029ce7b1f559ef5e747fcac439f1455a2ec7c5f09b72290795e70665044524e3e436f25df204ce27899af6ad638b44151bac5c3cf8a6a52116d1d659d8b000000001976a9145d50044454d3d18006e71b0da30d5c39b793053f88ac1935000000000000ffffffffdad860a5eede12e4b1e2e5596a2f0d8bfde2afa0448626ab562efde53d98507e0000000001000000


In [29]:
# hashing z 
z = double_sha256(unsigned_txn_1)
print(z)

9a0d8763bd0bd251683a3f06fdef9eb3981cba86afafc072d635f1cf3f13b160


In [30]:
calc_amount(2147483648)

'0000008000000000'

#  Testing 

In [158]:
txn_details = get_transaction_details('8801efe18da3d4f199a4a4e12450a66c60a60c208f7822701bbc7defb5ef0b0a', include_hex=True)

In [32]:
# Input hex string
hex_input = 'aa103ef423d7fef0539001670011c9f583d2c76e3fdfa9e2965f5d050afd1235'

# Convert hex string to bytes
bytes_data = bytes.fromhex(hex_input)

# Reverse the bytes
reversed_bytes = bytes_data[::-1]

# Convert reversed bytes back to hex
hex_output = reversed_bytes.hex()

# Print the results
print("Original Hex:", hex_input)
print("Reversed Bytes:", reversed_bytes)
print("Reversed Hex:", hex_output)


Original Hex: aa103ef423d7fef0539001670011c9f583d2c76e3fdfa9e2965f5d050afd1235
Reversed Bytes: b'5\x12\xfd\n\x05]_\x96\xe2\xa9\xdf?n\xc7\xd2\x83\xf5\xc9\x11\x00g\x01\x90S\xf0\xfe\xd7#\xf4>\x10\xaa'
Reversed Hex: 3512fd0a055d5f96e2a9df3f6ec7d283f5c9110067019053f0fed723f43e10aa


In [159]:
txn_details

{'block_hash': '00000000000000000001670a0549a38ecf2e51d40936c3cfdc2da4804e7b6e32',
 'block_height': 826109,
 'block_index': 3931,
 'hash': '8801efe18da3d4f199a4a4e12450a66c60a60c208f7822701bbc7defb5ef0b0a',
 'hex': '02000000000101524e3e436f25df204ce27899af6ad638b44151bac5c3cf8a6a52116d1d659d8b0000000000ffffffff02e803000000000000160014ffa40104eec28c18911c803f1207f8fb6d993fc819350000000000001600145d50044454d3d18006e71b0da30d5c39b793053f0247304402201626d2d8533e6f3f9150a88e565068037e012ba156ebd376666f05a9da1fb7e702206b37258e900391bc1f104ee4556830730a7d0291c5fb8f52eb15bcbbe9edbd62012103fb561eb15488bc679156766a55b7e6c5a382d6efbfe80cbe6c65ab05e75bc96600000000',
 'addresses': ['bc1q8z7vfvl5jk9z0vdckphf6wsdjfce6qhgskp84e',
  'bc1ql7jqzp8wc2xp3ygusql3yplcldkej07gk7ga9n',
  'bc1qt4gqg3z560gcqph8rvx6xr2u8xmexpfl0mt7mp'],
 'total': 14593,
 'fees': 6716,
 'size': 222,
 'vsize': 141,
 'preference': 'low',
 'relayed_by': '88.198.5.130:8333',
 'confirmed': datetime.datetime(2024, 1, 17, 9, 13, 38, tzin

In [148]:
calc_amount(9300)

'5424000000000000'

In [160]:
input_num = int(txn_details['hex'][12:14], 16)
print("Inputs :", input_num)

sig = txn_details["inputs"][0]["witness"][0]
sig_hex = hex(int(sig, 16))
print("Signature :", sig_hex[2:])

r_len = 2*int(sig_hex[8:10], 16)
r_end_idx = (10 + r_len)
r_ = sig_hex[10:r_end_idx]


s_len = 2*int(sig_hex[(r_end_idx + 2):(r_end_idx + 2 + 2)], 16)
s_end_idx = (r_end_idx + 2 + 2 + s_len)
s_ = sig_hex[(r_end_idx + 2 + 2):s_end_idx]

print("Length 'r' :", int(r_len/2))
print("r :", r_)

print("Length 's' :",int(s_len/2))
print("s :", s_)

Inputs : 1
Signature : 304402201626d2d8533e6f3f9150a88e565068037e012ba156ebd376666f05a9da1fb7e702206b37258e900391bc1f104ee4556830730a7d0291c5fb8f52eb15bcbbe9edbd6201
Length 'r' : 32
r : 1626d2d8533e6f3f9150a88e565068037e012ba156ebd376666f05a9da1fb7e7
Length 's' : 32
s : 6b37258e900391bc1f104ee4556830730a7d0291c5fb8f52eb15bcbbe9edbd62


In [161]:
ver = txn_details['hex'][0:8]
print(ver)

prev_out_start_idx = 14
prev_out_end_idx = prev_out_start_idx + 72 

seq_start_idx = prev_out_end_idx + 2
seq_end_idx = seq_start_idx + 8 

prev_outs_list = []
seq_list = []

for i in range(input_num):
    
    # fetching prev. outs and this list will contain outpoints of all segwit inputs
    prev_outs_list.append(txn_details['hex'][prev_out_start_idx:prev_out_end_idx]) 
    
    # fetching sequence
    seq_list.append(txn_details['hex'][seq_start_idx:seq_end_idx])
    
    # update indecies
    prev_out_start_idx = prev_out_start_idx + 10
    prev_out_end_idx = prev_out_start_idx + 72
    
    seq_start_idx = prev_out_end_idx + 2
    seq_end_idx = seq_start_idx + 8    
    
prev_outs  = ' '.join(prev_outs_list)
seq = ' '.join(seq_list)


print("Prevouts :", prev_outs)
print("Sequence :" ,seq)
#print(prev_outs_list)
#print(seq_list)

prev_outs_hash = double_sha256(prev_outs)
print("Prevouts hash  :", prev_outs_hash)

seq_hash = double_sha256(seq)
print("Sequence hash :", seq_hash)

#print(double_sha256('fff7f7881a8099afa6940d42d1e7f6362bec38171ea3edf433541db4e4ad969f00000000ef51e1b804cc89d182d279655c3aa89e815b1b309fe287d9b2b55d57b90ec68a01000000'))

02000000
Prevouts : 524e3e436f25df204ce27899af6ad638b44151bac5c3cf8a6a52116d1d659d8b00000000
Sequence : ffffffff
Prevouts hash  : 06f2b86854f8d3e7d86cb1e39c2b09919adde421fed7a141e6515ffd7f88a684
Sequence hash : 3bb13029ce7b1f559ef5e747fcac439f1455a2ec7c5f09b72290795e70665044


In [162]:
addr = address = txn_details["inputs"][0]["addresses"][0]
#addr = 'tb1qt9xzu0df95vsfal8eptzyruv4e00k4ty6d8zhh'
print("Address :", addr)

scriptcode = calc_scriptcode(addr)
print("Calculates script code:", scriptcode)

# write a loop to fetch all vals and keep it in a list or include this within the upper loop itself

output_amount = []
output_scriptcode = []
to_hash_list = []

for i in range(len(txn_details["outputs"])):
    amount = calc_amount(txn_details["outputs"][i]['value'])
    output_amount.append(amount)
    
    scriptcode_ = '16' + '00' + calc_scriptcode(txn_details["outputs"][i]["addresses"][0])[6:][:-4]
    output_scriptcode.append(scriptcode_)
                                 
    to_hash_list.append(amount)
    to_hash_list.append(scriptcode_)
    


to_hash = ''.join(to_hash_list)
to_hashoutputs = to_hash

print(to_hash_list)

print(to_hashoutputs)
hashoutputs = double_sha256(to_hashoutputs)

print("To hashoutput : ", to_hashoutputs)
print("hashoutputs  : ", hashoutputs)

locktime = txn_details['hex'][-8:]
print("Locktime", locktime)

sighash = '01000000'
print("Sighash :", sighash)

amount = calc_amount(txn_details['inputs'][0]['output_value'])
print('Amount', amount)

print(scriptcode)
print('28fddca5e7a484719bd96ac94c68c388bb02733166ec667b01988f56e1e78aa2')


Address : bc1q8z7vfvl5jk9z0vdckphf6wsdjfce6qhgskp84e
Calculates script code: 1976a91438bcc4b3f4958a27b1b8b06e9d3a0d92719d02e888ac
['e803000000000000', '160014ffa40104eec28c18911c803f1207f8fb6d993fc8', '1935000000000000', '1600145d50044454d3d18006e71b0da30d5c39b793053f']
e803000000000000160014ffa40104eec28c18911c803f1207f8fb6d993fc819350000000000001600145d50044454d3d18006e71b0da30d5c39b793053f
To hashoutput :  e803000000000000160014ffa40104eec28c18911c803f1207f8fb6d993fc819350000000000001600145d50044454d3d18006e71b0da30d5c39b793053f
hashoutputs  :  9e573421d9f32fc4f9997b707ddac0dd74d506a5295ab4c57af4608e29a0ae18
Locktime 00000000
Sighash : 01000000
Amount 3d53000000000000
1976a91438bcc4b3f4958a27b1b8b06e9d3a0d92719d02e888ac
28fddca5e7a484719bd96ac94c68c388bb02733166ec667b01988f56e1e78aa2


In [171]:
calc_scriptcode('bc1qfmsav54qgj8af49flwaymla7a56nv5gszss2u7')
calc_amount( 10169)

'b927000000000000'

In [163]:
print('nVersion:', ver)
print('hashPrevouts:', prev_outs_hash)
print('hashSequence:', seq_hash)
print('outpoint:', prev_outs_list[0])
print('scriptCode:', scriptcode)
print('amount:', amount)
print('nSequence:', seq)
print('hashOutputs:', hashoutputs)
print('nLockTime:', locktime)
print('nHashType:', sighash)

nVersion: 02000000
hashPrevouts: 06f2b86854f8d3e7d86cb1e39c2b09919adde421fed7a141e6515ffd7f88a684
hashSequence: 3bb13029ce7b1f559ef5e747fcac439f1455a2ec7c5f09b72290795e70665044
outpoint: 524e3e436f25df204ce27899af6ad638b44151bac5c3cf8a6a52116d1d659d8b00000000
scriptCode: 1976a91438bcc4b3f4958a27b1b8b06e9d3a0d92719d02e888ac
amount: 3d53000000000000
nSequence: ffffffff
hashOutputs: 9e573421d9f32fc4f9997b707ddac0dd74d506a5295ab4c57af4608e29a0ae18
nLockTime: 00000000
nHashType: 01000000


In [164]:
unsigned_txn_1 = ver + prev_outs_hash + seq_hash + prev_outs_list[0] + scriptcode + amount + seq + hashoutputs + locktime + sighash

In [165]:
print("Pre image :", unsigned_txn_1)
#0fee8ffb18a6e9315ea86a907fbde2b79257ed69beb878c753271aea94314613

Pre image : 0200000006f2b86854f8d3e7d86cb1e39c2b09919adde421fed7a141e6515ffd7f88a6843bb13029ce7b1f559ef5e747fcac439f1455a2ec7c5f09b72290795e70665044524e3e436f25df204ce27899af6ad638b44151bac5c3cf8a6a52116d1d659d8b000000001976a91438bcc4b3f4958a27b1b8b06e9d3a0d92719d02e888ac3d53000000000000ffffffff9e573421d9f32fc4f9997b707ddac0dd74d506a5295ab4c57af4608e29a0ae180000000001000000


In [166]:
unsigned_txn_hash = double_sha256(unsigned_txn_1)
print("Double hash of preimage :", unsigned_txn_hash)

Double hash of preimage : 988553b59f8f4befa0b4132cebeb1862f5055fae8cf63350c31437e6867d2e56


In [80]:
def decode_wif(wif_key):
    decoded_key = base58.b58decode(wif_key).hex()
    private_key_hex = decoded_key[2:-10]  # Remove version byte and checksum
    return private_key_hex

priv_key_wif = 'L1SaySfsFuEK9mHLLdcd9ygkqE5PenfbW6z9B6EuLR7CJ5ZWmkeU'
priv_key_hex = decode_wif(priv_key_wif)



# Ensure the private key has the correct length
if len(priv_key_hex) != 64:
    priv_key_hex = priv_key_hex.zfill(64)

print("Private key :", priv_key_hex)

Private key : 7df92bb9fca8cda5ed1fcbf1667515f026777414b2fdafddcf3115af5594e804


In [81]:
print(ver)
print(prev_outs_hash)
print(seq_hash)
print(prev_outs_list[0])
print(scriptcode)
print(amount)
print(seq)
print(hashoutputs)
print(locktime)
print(sighash)

02000000
06f2b86854f8d3e7d86cb1e39c2b09919adde421fed7a141e6515ffd7f88a684
3bb13029ce7b1f559ef5e747fcac439f1455a2ec7c5f09b72290795e70665044
524e3e436f25df204ce27899af6ad638b44151bac5c3cf8a6a52116d1d659d8b00000000
1976a91438bcc4b3f4958a27b1b8b06e9d3a0d92719d02e888ac
3d53000000000000
ffffffff
9e573421d9f32fc4f9997b707ddac0dd74d506a5295ab4c57af4608e29a0ae18
00000000
01000000


# Altered deterministic 'k' function and signing 

In [82]:
import embit

from binascii import b2a_base64
from hashlib import sha256

from embit import bip32, compact, ec
from embit.bip32 import HDKey
from embit.descriptor import Descriptor
from embit.networks import NETWORKS
from embit.util import secp256k1

In [97]:
def const_k_sig(z, pvt_key, pub_hash_k = None , z_int = None):
    p = 2**256 - 2**32 - 2**9 - 2**8 - 2**7 - 2**6 - 2**4 -1                               # proven prime number  
    n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141                 # number of points in the curve/field
    a, b = 0, 7                                                                            # y^2 = x^3 + ax + b -> secpk256k1 curve 

    g_x = 55066263022277343669578718895168534326250603453777594175500187360389116729240 
    g_y = 32670510020758816978083085130507043184471273380659243275938904335757337482424
    generator_pt = (g_x, g_y)                                                              # generator point 

    #all the required functions 
    def mod_inv(a,n = p): #Extended Euclidean Algorithm/'division' in elliptic curves
        lm, hm = 1,0
        low, high = a%n,n
        while low > 1:
            ratio = high//low
            nm, new = hm-lm*ratio, high-low*ratio
            lm, low, hm, high = nm, new, lm, low
        return lm % n

    def point_double(pt):
        lam = ((3*pt[0]**2 + a) * mod_inv(2*pt[1], p)) % p
    
        x = (lam**2 - 2*pt[0]) % p 
        y = (lam*(pt[0] - x) - pt[1]) % p
    
        return (x, y)


    def point_add(pt1, pt2):
        lam = ((pt2[1] - pt1[1]) * mod_inv((pt2[0] - pt1[0]), p)) % p
    
        x = (lam**2 - pt1[0] - pt2[0]) % p
        y = (lam*(pt1[0] - x) - pt1[1]) % p 
    
        return (x, y)


    # function to produce pub key from pvt key and generator pt 
    def point_mul(generator_pt, pvt_key_hex):
        if pvt_key_hex  == 0 or  pvt_key_hex >= n:
            raise Exception("Invalid Key")
        
        # pvt key in hex to binary 
        pvt_key_bin = str(bin(pvt_key_hex))[2:]
        r = generator_pt
    
        for i in range(1, len(pvt_key_bin)):
            # point double the pt for every bit in the pvt key in binary 
            r = point_double(r)
        
            # if bit = 1, then both point double & point add
            if pvt_key_bin[i] == '1':
                r = point_add(r, generator_pt)
            
            # return this new pt 'r' which is the pub key || pub key = pvt key * generator pt       
        
        return r  

    # defining constants
    pub_key = point_mul(generator_pt, int.from_bytes(pvt_key, 'big'))    # uncompressed pub key, i.e, the product of point mul
    print(pub_key)
    
    k = deterministic_k(int(pub_hash_k), z_int)
    #k = 3
    
    g = generator_pt
    x_r, y_r = point_mul(g, k)
    
    r = x_r % n
    z = int.from_bytes(z[:32], 'big')
    s = (((z + x_r*int.from_bytes(pvt_key, 'big')) % n) * mod_inv(k, n)) % n    # regular math

    return r.to_bytes(32, "little") + s.to_bytes(32, "little")

In [84]:
bip39_seed = '3a11298f7bf61605c6740a44aced41a547069298a0c971d7a0fd69c3c0395a21af3b6c937e700a336457edd67a6eacee6928259082304ad8adecb1a2560781a9'
bip39_seed_bytes = bytes.fromhex(bip39_seed)

print(bip39_seed_bytes)

b':\x11)\x8f{\xf6\x16\x05\xc6t\nD\xac\xedA\xa5G\x06\x92\x98\xa0\xc9q\xd7\xa0\xfdi\xc3\xc09Z!\xaf;l\x93~p\n3dW\xed\xd6zn\xac\xeei(%\x90\x820J\xd8\xad\xec\xb1\xa2V\x07\x81\xa9'


In [85]:
embit_network = 'main'
derivation = "m/84'/0'/0'/0/9"

In [86]:
seed_bytes = bip39_seed_bytes
root = bip32.HDKey.from_seed(seed_bytes, version=NETWORKS[embit_network]["xprv"])
prv = root.derive(derivation).key

In [87]:
prv_bytes = prv.secret    # prv.secret by default is in bytes we'll be passing this as i/p to our signing function 
pub = prv.get_public_key()
#pub = py_secp256k1.ec_pubkey_create(prv._secret)
pub_str = str(pub)
pub_bytes = bytes.fromhex(pub_str)
#pub_hash = hashlib.sha256(pub).digest().hex()

pub_hash = sha256(pub_bytes).digest().hex()  # we'll be using this to replace private key while generating deterministic 'k'


#pub_key_hash = sha256(pub)
#print(pub_hex)

print("Private key :", prv)
print("Public key :", pub)
print("Public key hash :", pub_hash) 

Private key : L1SaySfsFuEK9mHLLdcd9ygkqE5PenfbW6z9B6EuLR7CJ5ZWmkeU
Public key : 03fb561eb15488bc679156766a55b7e6c5a382d6efbfe80cbe6c65ab05e75bc966
Public key hash : fde86c7b6f53c3902a844c9ff4117aae215394bd7b63cba0becfe808ee448a15


In [79]:
print(prv_bytes)

b'}\xf9+\xb9\xfc\xa8\xcd\xa5\xed\x1f\xcb\xf1fu\x15\xf0&wt\x14\xb2\xfd\xaf\xdd\xcf1\x15\xafU\x94\xe8\x04'


In [77]:
hashlib.sha256(py_secp256k1.ec_pubkey_create(prv._secret)).digest().hex()

'ed8f0b81790d8b3f786887748bd54943dc10be2b3a68607cac87f7c2a8ad51de'

In [70]:
py_secp256k1.ec_pubkey_create(bytes.fromhex('efd228cd9aa02c990409be22cbe9c194b1848cbbf6c36269b107ce9fa6807b12')).hex()

'f0705a928f3bc2bf8ac1efd488df745814c5e473827752765cedfbcffcbbf9a5873cfdc717e33d506c0a35c170a8184d263af4524237df4e38d015b07dfa4267'

In [101]:
pub_hash = int.from_bytes(hashlib.sha256(py_secp256k1.ec_pubkey_create(prv._secret)).digest(), 'little')
print(pub_hash)

100557765549653634707878779641659417603050102636040280366749842585400482959341


In [77]:
bytes.fromhex(py_secp256k1.ec_pubkey_create(prv._secret).hex())

b'f\xc9[\xe7\x05\xabel\xbe\x0c\xe8\xbf\xef\xd6\x82\xa3\xc5\xe6\xb7UjvV\x91g\xbc\x88T\xb1\x1eV\xfb\x15b\xf86\xc7Z\xf1\x0f\xc7\xaeMP\xca\xba{u\x1dM\xb0]\xdc \x99b\xdb\xa2rz\xe1\xaaBN'

In [98]:
import hmac 

SECP256K1_ORDER = n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141

def deterministic_k(secret, z, extra_data=None):
    # RFC6979, optimized for secp256k1
    k = b"\x00" * 32
    v = b"\x01" * 32
    if z > SECP256K1_ORDER:
        z -= SECP256K1_ORDER
    z_bytes = z.to_bytes(32, "big")
    secret_bytes = secret.to_bytes(32, "big") #because sha256(pub_key is always in bytes)
    if extra_data is not None:
        z_bytes += extra_data
    k = hmac.new(k, v + b"\x00" + secret_bytes + z_bytes, "sha256").digest()
    v = hmac.new(k, v, "sha256").digest()
    k = hmac.new(k, v + b"\x01" + secret_bytes + z_bytes, "sha256").digest()
    v = hmac.new(k, v, "sha256").digest()
    while True:
        v = hmac.new(k, v, "sha256").digest()
        candidate = int.from_bytes(v, "big")
        if candidate >= 1 and candidate < SECP256K1_ORDER:
            return candidate
        k = hmac.new(k, v + b"\x00", "sha256").digest()
        v = hmac.new(k, v, "sha256").digest()

In [104]:
pub_hash_bytes =  bytes.fromhex(pub_hash) # pub hash in bytes will be going in the 'k' deterministic function 

pre_image_hash = '988553b59f8f4befa0b4132cebeb1862f5055fae8cf63350c31437e6867d2e56'
pre_image_hash_bytes = bytes.fromhex(pre_image_hash)    # we'll be passing this as input to the signing function

pre_image_hash_int = int.from_bytes(pre_image_hash_bytes, "big") # this will be going as digest to the det 'k' same function

In [105]:
k = deterministic_k(int.from_bytes(prv_bytes, 'big'), pre_image_hash_int)
print(k)

91286349473670812610449901780980074302206045927493130326279076863825416091182


In [119]:
prv_int = int.from_bytes(prv_bytes, 'big')

pub_hash_int = int.from_bytes(pub_hash_bytes, 'big')


signature = '304402201626d2d8533e6f3f9150a88e565068037e012ba156ebd376666f05a9da1fb7e702206b37258e900391bc1f104ee4556830730a7d0291c5fb8f52eb15bcbbe9edbd6201'

r_len = 2*int(signature[6:8], 16)
r_hex = signature[8:(8+r_len)]

s_len = 2*int(signature[(8+r_len+2):(8+r_len+2+2)], 16)
s_hex = signature[(8+r_len+2+2):(8+r_len+2+2+s_len)]

r_int = int(r_hex, 16)
s_int = int(s_hex, 16)

k = deterministic_k(prv_int, pre_image_hash_int)



#sig = const_k_sig(pre_image_hash_bytes, prv_bytes, prv_int, pre_image_hash_int)



#r_bytes = sig[:32]
#s_bytes = sig[32:64]
#print(sig)

#r_int = int.from_bytes(r_bytes, 'little')
#s_int = int.from_bytes(s_bytes, 'little')

#k = deterministic_k(prv_int, pre_image_hash_int)
#print(k)

In [110]:
sig

b'\xe3\xaa`\xb9K\xc3\xfb3\x0e\xb8$\xb7&\xc6C\xe3d\xc9\x06\xaa\xfc)\x84a\xa7\xf1\xe6\x83\xb9\xbb\x0bP:=\xed\xeb\xa4\x1a\x97@\x85\xf3\xdd \xe2\xced\xb12\x00WT\xd0\xcd#7n\xb8\x98b\xe1\x05\x9e\x95'

In [150]:
print(0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141)

115792089237316195423570985008687907852837564279074904382605163141518161494337


In [151]:
int.from_bytes(pub_hash_bytes, "big") < SECP256K1_ORDER

True

# Calculating private key from signature 

In [116]:
p = 2**256 - 2**32 - 2**9 - 2**8 - 2**7 - 2**6 - 2**4 -1 

def mod_inv(a,n = p): #Extended Euclidean Algorithm/'division' in elliptic curves
    lm, hm = 1,0
    low, high = a%n,n
    
    while low > 1:
        ratio = high//low
        nm, new = hm-lm*ratio, high-low*ratio
        lm, low, hm, high = nm, new, lm, low
        
    return lm % n

### This is how 's' is calculated 
s = (((z + x_r*int.from_bytes(pvt_key, 'big')) % n) * mod_inv(k, n)) % n    `# regular math

In [120]:
# to calculate pvt. key we already know r, s and k 
calc_prv = (mod_inv(r_int, n)*(k*s_int - pre_image_hash_int)) % n 

In [121]:
calc_prv_hex = hex(calc_prv)
print(calc_prv_hex)
print(priv_key_hex)

print(calc_prv_hex[2:] == priv_key_hex)

0x9979a5494da74d35e2bf075bb4bbda2ecb0e08c94cfb22499330ee3d73383846
7df92bb9fca8cda5ed1fcbf1667515f026777414b2fdafddcf3115af5594e804
False


In [97]:
print(prv_bytes)

b'}\xf9+\xb9\xfc\xa8\xcd\xa5\xed\x1f\xcb\xf1fu\x15\xf0&wt\x14\xb2\xfd\xaf\xdd\xcf1\x15\xafU\x94\xe8\x04'


In [98]:
original_sig_seedsigner_sig = secp256k1.ecdsa_sign_recoverable(pre_image_hash_bytes, prv_bytes) 
print(original_sig_seedsigner_sig)

b'}\x0fL\x10\xbc\x97\xde\x10:\x98;\x83\x8ak\xb5\xdb\x8b\xe2\xee\xa2m\x80\xdc3K\xe2CX\x84\r9\xe1\xad\xfd\xe9\xa9*\x1f\rq\xbe3\xd4\xad\xe0j\xc5s3H\\\xafj\xb4\x8c\xf4\xac\x0b\xf5\x15\xf7\xf0\xea*\x00'


In [195]:
pub = py_secp256k1.ec_pubkey_create(prv._secret)
print(len(pub.hex()))

128


In [196]:
print(pub_hash_bytes.hex())

sig = const_k_sig(pre_image_hash_bytes, prv_bytes, pub_hash_bytes, pre_image_hash_int)
#print(sig)

pub = py_secp256k1.ec_pubkey_create(prv._secret)
lk = py_secp256k1.ecdsa_verify(sig, pre_image_hash_bytes, pub)
print(lk)

#print(sig.hex())



for i in range(4):
    if py_secp256k1.ecdsa_recover(sig + bytes([i]), pre_image_hash_bytes) == pub:
        sig = sig + bytes([i])
        break
    
malicious_sig = sig
#print(malicious_sig)
    
#print(original_sig_seedsigner_sig == malicious_sig)
#

346598286f5a0a3a2b7f42ac31dab9bd9dee3d30db1f970cda6dd5059935d6da
(75072862377745462948128548761028566339551352998868500282649073297423951622384, 46706564135475779232138557665644278139383549497825509258258125466601992830087)
False


ValueError: Failed to recover public key

In [197]:
r_int = int.from_bytes(sig[:32], "big")
s_int = int.from_bytes(sig[:32], "big")


In [158]:
deterministic_k(prv_bytes, msghash_int)

19733093509415893865808793833768214963321975555026301162357824588355781975809

In [159]:
msghash = pre_image_hash_bytes

In [179]:
 ## add a malicious signing logic
prv_bytes = prv.secret    # prv.secret by default is in bytes we'll be passing this as i/p to our signing function 
pub = prv.get_public_key()
pub_str = str(pub)
pub_bytes = bytes.fromhex(pub_str)

pub_hash = sha256(pub_bytes).digest().hex()
pub_hash_bytes = bytes.fromhex(pub_hash) # pub hash in bytes will be going in the 'k' deterministic function 

msghash_int = int.from_bytes(msghash, "big")

sig = const_k_sig(msghash, prv_bytes, prv_bytes, msghash_int)
pub = py_secp256k1.ec_pubkey_create(prv._secret)

print(sig)
print(len(sig))

for i in range(4):
    if py_secp256k1.ecdsa_recover(sig + bytes([i]), msghash) == pub:
        sig = sig + bytes([i])
        break
    
print()

(75072862377745462948128548761028566339551352998868500282649073297423951622384, 46706564135475779232138557665644278139383549497825509258258125466601992830087)
b'\xe7\x95\xf9\xa8\xcc\xb1\xb4<+\xf2\x9e\x86\xdfS\xdb\xc7\x16i\xf29Mk?\xc0\xeb2\x9f)\x86\xc6\xbb2\xb7\x8bE2k\x82"\xeb\xf8\x9f\xfa\xffa\xa8;\x91\x0c\x94\xf3z\x8ah\x12\xf36\xfd7\x04\x9b\xfb\xa5I'
64


In [164]:
sig

b'\xe7\x95\xf9\xa8\xcc\xb1\xb4<+\xf2\x9e\x86\xdfS\xdb\xc7\x16i\xf29Mk?\xc0\xeb2\x9f)\x86\xc6\xbb2\xb7\x8bE2k\x82"\xeb\xf8\x9f\xfa\xffa\xa8;\x91\x0c\x94\xf3z\x8ah\x12\xf36\xfd7\x04\x9b\xfb\xa5I\x00'

In [165]:
r = sig[:32]
s = sig[32:64]

r_int = int.from_bytes(r_bytes, 'little')
s_int = int.from_bytes(s_bytes, 'little')
print(s_int)

110965473006965171459282381957884705940765335269367368361253915592668877477108


In [166]:
int.from_bytes(r_bytes, 'little') == int.from_bytes(r_bytes, 'big')

False

In [167]:
s_inv = mod_inv(s_int, n)
print(s_inv)

8018832574689321946771824711586173263995065701448906809015844109504961581514


In [168]:
pub_int = int.from_bytes(pub_bytes, 'big')
prv_int = int.from_bytes(prv_bytes, 'big')

In [112]:
r_times_pub = (r_int*pub_int)%n

In [113]:
r_times_prv = (r_int*prv_int)%n

In [169]:
def point_mul(generator_pt, pvt_key_hex):
    if pvt_key_hex  == 0 or  pvt_key_hex >= n:
        raise Exception("Invalid Key")
        
    # pvt key in hex to binary 
    pvt_key_bin = str(bin(pvt_key_hex))[2:]
    r = generator_pt
    
    for i in range(1, len(pvt_key_bin)):
        # point double the pt for every bit in the pvt key in binary 
        r = point_double(r)
        
        # if bit = 1, then both point double & point add
        if pvt_key_bin[i] == '1':
            r = point_add(r, generator_pt)
            
            
    # return this new pt 'r' which is the pub key || pub key = pvt key * generator pt        
    return r   

In [170]:
def sig_verify(generator_pt, k, z, r, s):
    w = mod_inv(s, n) 
    
    u1 = point_mul(generator_pt, (z * w) % n) 
    u2 = point_mul(pub_key, (r * w) % n) 
    
    x_r2, y_r2 = point_add(u1, u2)

    print("\nSignature Verification", r == x_r2)

In [171]:
g_x = 55066263022277343669578718895168534326250603453777594175500187360389116729240 
g_y = 32670510020758816978083085130507043184471273380659243275938904335757337482424
generator_pt = (g_x, g_y)

z = pre_image_hash
print(r_int)
print(s_int)

sig_verify(generator_pt, k, z, r_int, s_int)

14064726522498138637560722988653466761618556537288212226644608271927716912741
110965473006965171459282381957884705940765335269367368361253915592668877477108


OverflowError: cannot fit 'int' into an index-sized integer

In [95]:
def ecdsa_signature_serialize_der(sig, context=None):
    print(len(sig))
    if len(sig) != 64:
        raise ValueError("Signature should be 64 bytes long")
    r = int.from_bytes(sig[:32], "little")
    s = int.from_bytes(sig[32:], "little")
    rb = r.to_bytes((r.bit_length() + 8) // 8, "big")
    sb = s.to_bytes((s.bit_length() + 8) // 8, "big")
    return (
        b"\x30"
        + bytes([4 + len(rb) + len(sb), 2, len(rb)])
        + rb
        + bytes([2, len(sb)])
        + sb
    )

sig_der = ecdsa_signature_serialize_der(sig[0:64])
print(sig_der)



64
b'0D\x02 2\xbb\xc6\x86)\x9f2\xeb\xc0?kM9\xf2i\x16\xc7\xdbS\xdf\x86\x9e\xf2+<\xb4\xb1\xcc\xa8\xf9\x95\xe7\x02 I\xa5\xfb\x9b\x047\xfd6\xf3\x12h\x8az\xf3\x94\x0c\x91;\xa8a\xff\xfa\x9f\xf8\xeb"\x82k2E\x8b\xb7'


In [145]:
sig = sig_der

if sig[1] + 2 != len(sig):
    print('False')
if len(sig) < 4:
    print('False')
if sig[0] != 0x30:
    print('False')
if sig[2] != 0x02:
    print('False')
rlen = sig[3]
if len(sig) < 6 + rlen:
    print('False')
if rlen < 1 or rlen > 33:
    print('False')
if sig[4] >= 0x80:
    print('False')
if rlen > 1 and (sig[4] == 0) and not (sig[5] & 0x80):
    print('False')
r = int.from_bytes(sig[4 : 4 + rlen], "big")
if sig[4 + rlen] != 0x02:
    print('False')
slen = sig[5 + rlen]
if slen < 1 or slen > 33:
    print('False')
if len(sig) != 6 + rlen + slen:
    print('False')
if sig[6 + rlen] >= 0x80:
    print('False')
if slen > 1 and (sig[6 + rlen] == 0) and not (sig[7 + rlen] & 0x80):
    print('False')
s = int.from_bytes(sig[6 + rlen : 6 + rlen + slen], "big")


print(r)

22947412993069689809413313218414555297919684427191669171944219326559928554983


In [111]:
SECP256K1_FIELD_SIZE = 2**256 - 2**32 - 977
SECP256K1 = EllipticCurve(SECP256K1_FIELD_SIZE, 0, 7)
SECP256K1_G = (
    0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798,
    0x483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8,
    1,
)
SECP256K1_ORDER = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141
SECP256K1_ORDER_HALF = SECP256K1_ORDER // 2

low_s=True

if r < 1 or s < 1 or r >= SECP256K1_ORDER or s >= SECP256K1_ORDER:
    print('False')
if low_s and s >= SECP256K1_ORDER_HALF:
    print('False')
z = int.from_bytes(msghash, "big")

In [112]:
def modinv(a, n):
    """Compute the modular inverse of a modulo n using the extended Euclidean
    Algorithm. See https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm#Modular_integers.
    """
    # TODO: Change to pow(a, -1, n) available in Python 3.8
    t1, t2 = 0, 1
    r1, r2 = n, a
    while r2 != 0:
        q = r1 // r2
        t1, t2 = t2, t1 - q * t2
        r1, r2 = r2, r1 - q * r2
    if r1 > 1:
        return None
    if t1 < 0:
        t1 += n
    return t1

In [113]:
 def affine(self, p1):
        """Convert a Jacobian point tuple p1 to affine form, or None if at infinity.

        An affine point is represented as the Jacobian (x, y, 1)"""
        x1, y1, z1 = p1
        if z1 == 0:
            return None
        inv = modinv(z1, self.p)
        inv_2 = (inv**2) % self.p
        inv_3 = (inv_2 * inv) % self.p
        return ((inv_2 * x1) % self.p, (inv_3 * y1) % self.p, 1)

In [117]:
w = modinv(s, SECP256K1_ORDER)
u1 = z * w % SECP256K1_ORDER
u2 = r * w % SECP256K1_ORDER

print(u1)
print(u2)

R = SECP256K1.affine(SECP256K1.mul([(SECP256K1_G, u1), (self.p, u2)]))

if R is None or (R[0] % SECP256K1_ORDER) != r:
    print('False')
print('True')

63564039184791089413261167011382132900511655760780495071022540642697924649285
55374581975413887350269077605189805099828903790087199534783532339441190465288


NameError: name 'self' is not defined

In [69]:
py_secp256k1.ecdsa_recover(sig + bytes([i]), msghash)

ValueError: Failed to recover public key

In [71]:
ecdsa_verify(sig[:64], msghash, pub)

TypeError: ecdsa_verify() missing 1 required positional argument: 'coin'

In [89]:
if not False:
    print('hi')

hi


# Signature  

In [15]:
print("Address :", addr)
print("Public key : 03a5f9bbfccffbed5c7652778273e4c5145874df88d4efc18abfc23b8f925a70f0")
print("Private key : L5Ftf4S3FUPU5ePS4uzgnXQqeuo2mgHnvAAvJKxoAvpaizC5CJt5")

Address : bc1q777gx7q5lmexu3feceph3yuc9zyresd5fejjt3
Public key : 03a5f9bbfccffbed5c7652778273e4c5145874df88d4efc18abfc23b8f925a70f0
Private key : L5Ftf4S3FUPU5ePS4uzgnXQqeuo2mgHnvAAvJKxoAvpaizC5CJt5


Private key : efd228cd9aa02c990409be22cbe9c194b1848cbbf6c36269b107ce9fa6807b12


In [172]:
sk = ecdsa.SigningKey.from_string(bytes.fromhex(priv_key_hex), curve=ecdsa.SECP256k1, hashfunc=hashlib.sha256)

print(sk)

unsigned_txn = unsigned_txn_1
#hashed_message = double_sha256(unsigned_txn)
#print(hashed_message)

#signature = sk.sign_deterministic(bytes(hashed_message, 'utf-8'), hashfunc=hashlib.sha256)
#signature = sk.sign_deterministic(bytes(unsigned_txn, 'utf-8'), hashfunc=hashlib.sha256)

#signature = sk.sign_deterministic(bytes.fromhex(unsigned_txn), hashfunc=hashlib.sha256, sigencode=sigencode_der)

#unsigned_txn_single_hash = 'c304d56804b24a6801a77803281a497f5526e20f14e65df1006887fc57f0ee39'

unsigned_txn_double_hash = double_sha256(unsigned_txn)
print("Digest (double SHA) :", unsigned_txn_double_hash)

#signature = sk.sign_deterministic(bytes.fromhex(unsigned_txn_single_hash), hashfunc=hashlib.sha256)
signature = sk.sign_digest_deterministic(bytes.fromhex(unsigned_txn_double_hash), hashfunc=hashlib.sha256)

signature = signature.hex()
print(len(signature))
print(signature[:64])
print(signature[64:128])
#print('3044022032bbc686299f32ebc03f6b4d39f26916c7db53df869ef22b3cb4b1cca8f995e7022049a5fb9b0437fd36f312688a7af3940c913ba861fffa9ff8eb22826b32458bb701')


Digest (double SHA) : 523a1837aace053c4bbb61ee260f48493e14bb46f452eaafe533b6bcd4385dcf
128
32bbc686299f32ebc03f6b4d39f26916c7db53df869ef22b3cb4b1cca8f995e7
49a5fb9b0437fd36f312688a7af3940c913ba861fffa9ff8eb22826b32458bb7


In [177]:
vk = sk.get_verifying_key()
vk.verify(sig[:64], pre_image_hash_bytes) # True

BadSignatureError: Signature verification failed

In [178]:
sig

b'\xe7\x95\xf9\xa8\xcc\xb1\xb4<+\xf2\x9e\x86\xdfS\xdb\xc7\x16i\xf29Mk?\xc0\xeb2\x9f)\x86\xc6\xbb2\xb7\x8bE2k\x82"\xeb\xf8\x9f\xfa\xffa\xa8;\x91\x0c\x94\xf3z\x8ah\x12\xf36\xfd7\x04\x9b\xfb\xa5I\x00'

In [173]:
#converting WIF to private key 

pvt_key_wif = 'L5Ftf4S3FUPU5ePS4uzgnXQqeuo2mgHnvAAvJKxoAvpaizC5CJt5'

first_encode = base58.b58decode(pvt_key_wif)
print(first_encode)
print(first_encode.hex())

pvt_key = first_encode.hex()
pvt_key = pvt_key[2:][:-8][:-2]
print(pvt_key)

b'\x80\xef\xd2(\xcd\x9a\xa0,\x99\x04\t\xbe"\xcb\xe9\xc1\x94\xb1\x84\x8c\xbb\xf6\xc3bi\xb1\x07\xce\x9f\xa6\x80{\x12\x01R\xc0B\x8e'
80efd228cd9aa02c990409be22cbe9c194b1848cbbf6c36269b107ce9fa6807b120152c0428e
efd228cd9aa02c990409be22cbe9c194b1848cbbf6c36269b107ce9fa6807b12


In [25]:
priv_key_hex = '619c335025c7f4012e556c2a58b2506e30b8511b53ade95ea316fd8c3286feb9'
sk = ecdsa.SigningKey.from_string(bytes.fromhex(priv_key_hex), curve=ecdsa.SECP256k1, hashfunc=hashlib.sha256)

print(sk)

unsigned_txn = '0100000096b827c8483d4e9b96712b6713a7b68d6e8003a781feba36c31143470b4efd3752b0a642eea2fb7ae638c36f6252b6750293dbe574a806984b8e4d8548339a3bef51e1b804cc89d182d279655c3aa89e815b1b309fe287d9b2b55d57b90ec68a010000001976a9141d0f172a0ecb48aee1be1f2687d2963ae33f71a188ac0046c32300000000ffffffff863ef3e1a92afbfdb97f31ad0fc7683ee943e9abcf2501590ff8f6551f47e5e51100000001000000'
#hashed_message = double_sha256(unsigned_txn)
#print(hashed_message)

#signature = sk.sign_deterministic(bytes(hashed_message, 'utf-8'), hashfunc=hashlib.sha256)
#signature = sk.sign_deterministic(bytes(unsigned_txn, 'utf-8'), hashfunc=hashlib.sha256)

#signature = sk.sign_deterministic(bytes.fromhex(unsigned_txn), hashfunc=hashlib.sha256, sigencode=sigencode_der)

#unsigned_txn_single_hash = 'c304d56804b24a6801a77803281a497f5526e20f14e65df1006887fc57f0ee39'

unsigned_txn_double_hash = double_sha256(unsigned_txn)
print(unsigned_txn_double_hash)

#signature = sk.sign_deterministic(bytes.fromhex(unsigned_txn_single_hash), hashfunc=hashlib.sha256)
signature = sk.sign_digest_deterministic(bytes.fromhex(unsigned_txn_double_hash))

signature = signature.hex()
print(len(signature))
print(signature[:64])
print(signature[64:128])
print('304402203609e17b84f6a7d30c80bfa610b5b4542f32a8a0d5447a12fb1366d7f01cc44a0220573a954c4518331561406f90300e8f3358f51928d43c212a8caed02de67eebee')


c37af31116d1b27caf68aae9e3ac82f1477929014d5b917657d0eb49478cb670
c37af31116d1b27caf68aae9e3ac82f1477929014d5b917657d0eb49478cb670
128
3609e17b84f6a7d30c80bfa610b5b4542f32a8a0d5447a12fb1366d7f01cc44a
a8c56ab3bae7ccea9ebf906fcff170cb61b9c3bddb0c7f1133238e5ee9b75553
304402203609e17b84f6a7d30c80bfa610b5b4542f32a8a0d5447a12fb1366d7f01cc44a0220573a954c4518331561406f90300e8f3358f51928d43c212a8caed02de67eebee


# Garbage

In [ ]:
"02000000000101

In [55]:
import ecdsa
from ecdsa import SECP256k1
import hashlib

def sign_transaction(private_key, unsigned_transaction_hex):
    # Deserialize the unsigned transaction hex
    unsigned_transaction_bytes = bytes.fromhex(unsigned_transaction_hex)

    # Hash the transaction
    transaction_hash = double_sha256(unsigned_transaction_bytes)

    # Create an ECDSA signature
    signature = ecdsa.SigningKey.from_string(private_key, curve=SECP256k1).sign_digest(transaction_hash, sigencode=ecdsa.util.sigencode_der_canonize)

    # Verify the signature (optional, for demonstration purposes)
    verification_key = ecdsa.VerifyingKey.from_string(ecdsa.SigningKey.from_string(private_key, curve=SECP256k1).get_verifying_key().to_string(), curve=SECP256k1)
    if not verification_key.verify_digest(signature, transaction_hash, sigdecode=ecdsa.util.sigdecode_der):
        raise ValueError("Signature verification failed.")

    # Return the signature as bytes
    return signature

def double_sha256(data):
    return hashlib.sha256(hashlib.sha256(data).digest()).digest()

# Example Usage
private_key_hex = "26F85CE8B2C635AD92F6148E4443FE415F512F3F29F44AB0E2CBDA819295BBD5"  # Replace with your actual private key in hexadecimal format
unsigned_transaction_hex = "0200000099197e88ff743aff3e453e3a7b745abd31937ccbd56f96a179266eba786833e682a7d5bb59fc957ff7f737ca0b8be713c705d6173783ad5edb067819bed70be89cb872539fbe1bc0b9c5562195095f3f35e6e13919259956c6263c9bd53b20b7010000001976a914594c2e3da92d1904f7e7c856220f8cae5efb556488ac5424000000000000fffffffff3ae23c3fd63a2e0479888f95c7a8ab221b20add6ac819e9d8953edd1a0cd9240000000001000000"

private_key = bytes.fromhex(private_key_hex)
signature = sign_transaction(private_key, unsigned_transaction_hex)
print("ECDSA Signature:", signature.hex())


ECDSA Signature: 304402203763c5cba2af1afb499efcd226017a96b4c6a45453bc90be4e82a653f69102660220418b0aa7d9981d4e3dc376687fa7c528e8c40a22887f240ee96b96a8acc1ae87


In [58]:
import ecdsa
from ecdsa import SECP256k1
import hashlib
import hmac

def deterministic_sign(private_key_hex, unsigned_transaction_hex):
    # Convert private key to bytes
    private_key = bytes.fromhex(private_key_hex)

    # Deserialize the unsigned transaction hex
    unsigned_transaction_bytes = bytes.fromhex(unsigned_transaction_hex)

    # Hash the transaction
    transaction_hash = double_sha256(unsigned_transaction_bytes)

    # Create an ECDSA private key
    signing_key = ecdsa.SigningKey.from_string(private_key, curve=SECP256k1)

    # RFC 6979 deterministic k generation using HMAC-SHA256
    deterministic_k = ecdsa.rfc6979.generate_k(signing_key.curve.order, signing_key.privkey.secret_multiplier, transaction_hash, hmac.new, hashlib.sha256).digest()

    # Sign the hash using the deterministic k
    signature = signing_key.sign_digest_deterministic(transaction_hash, hashfunc=hashlib.sha256, sigencode=ecdsa.util.sigencode_der, k=deterministic_k)

    # Return the signature as bytes
    return signature

def double_sha256(data):
    return hashlib.sha256(hashlib.sha256(data).digest()).digest()

# Example Usage
private_key_hex = "26F85CE8B2C635AD92F6148E4443FE415F512F3F29F44AB0E2CBDA819295BBD5"
unsigned_transaction_hex = "0200000099197e88ff743aff3e453e3a7b745abd31937ccbd56f96a179266eba786833e682a7d5bb59fc957ff7f737ca0b8be713c705d6173783ad5edb067819bed70be89cb872539fbe1bc0b9c5562195095f3f35e6e13919259956c6263c9bd53b20b7010000001976a914594c2e3da92d1904f7e7c856220f8cae5efb556488ac5424000000000000fffffffff3ae23c3fd63a2e0479888f95c7a8ab221b20add6ac819e9d8953edd1a0cd9240000000001000000"

signature = deterministic_sign(private_key_hex, unsigned_transaction_hex)
print("ECDSA Signature:", signature.hex())


TypeError: 'bytes' object is not callable

In [62]:
import ecdsa
from hashlib import sha256

# SECP256k1 is the Bitcoin elliptic curve
sk = "26F85CE8B2C635AD92F6148E4443FE415F512F3F29F44AB0E2CBDA819295BBD5"#ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1, hashfunc=sha256) # The default is sha1
hash_ = "4876161197833dd58a1a2ba20728633677f38b9a7513a4d7d3714a7f7d3a1fa2"
sig = sk.sign(b"4876161197833dd58a1a2ba20728633677f38b9a7513a4d7d3714a7f7d3a1fa2")


AttributeError: 'str' object has no attribute 'sign'

In [63]:
!pip install starkbank-ecdsa

In [65]:
# Generate Keys
privateKey = PrivateKey()
publicKey = privateKey.publicKey()

print(publicKey)

message = "My test message"

# Generate Signature
signature = Ecdsa.sign(message, privateKey)

# Verify if signature is valid
print (Ecdsa.verify(message, signature, publicKey))

NameError: name 'PrivateKey' is not defined

In [74]:
import ecdsa
from ecdsa import SECP256k1
import hashlib

def sign_transaction(private_key, unsigned_transaction_hex):
    # Deserialize the unsigned transaction hex
    unsigned_transaction_bytes = bytes.fromhex(unsigned_transaction_hex)

    # Hash the transaction
    transaction_hash = double_sha256(unsigned_transaction_bytes)

    # Create an ECDSA signature
    signature = ecdsa.SigningKey.from_string(private_key, curve=SECP256k1).sign_digest(transaction_hash, sigencode=ecdsa.util.sigencode_der_canonize)

    # Verify the signature (optional, for demonstration purposes)
    verification_key = ecdsa.VerifyingKey.from_string(ecdsa.SigningKey.from_string(private_key, curve=SECP256k1).get_verifying_key().to_string(), curve=SECP256k1)
    if not verification_key.verify_digest(signature, transaction_hash, sigdecode=ecdsa.util.sigdecode_der):
        raise ValueError("Signature verification failed.")

    # Return the signature as bytes
    return signature

def double_sha256(data):
    return hashlib.sha256(hashlib.sha256(data).digest()).digest()

# Example Usage
private_key_hex = "26F85CE8B2C635AD92F6148E4443FE415F512F3F29F44AB0E2CBDA819295BBD5"  # Replace with your actual private key in hexadecimal format
unsigned_transaction_hex = "0200000099197e88ff743aff3e453e3a7b745abd31937ccbd56f96a179266eba786833e682a7d5bb59fc957ff7f737ca0b8be713c705d6173783ad5edb067819bed70be89cb872539fbe1bc0b9c5562195095f3f35e6e13919259956c6263c9bd53b20b7010000001976a914594c2e3da92d1904f7e7c856220f8cae5efb556488ac5424000000000000fffffffff3ae23c3fd63a2e0479888f95c7a8ab221b20add6ac819e9d8953edd1a0cd9240000000001000000"

private_key = bytes.fromhex(private_key_hex)
signature = sign_transaction(private_key, unsigned_transaction_hex)
print("Code from chatGPT :", signature.hex())


Code from chatGPT : 3044022025ad334a7d3b691109168651ab163c369e684314bb350d9000028034b9201a1502202371e952e88e9ab9f290cb776b109cc89ec97a9041a780fbfeaf450a95a1211e


In [72]:
print('Code from the internet and then encoded by chatGPT : 302202201b68854b376516bca9d8f342d1f33b09dc0e4acc8c41d3cdf2345a4352bcd8c7022167763fd346723ca063bfee413fbc10f9b074fda1666f0839f9f202b83a57c3a416')

Code from the internet and then encoded by chatGPT : 302202201b68854b376516bca9d8f342d1f33b09dc0e4acc8c41d3cdf2345a4352bcd8c7022167763fd346723ca063bfee413fbc10f9b074fda1666f0839f9f202b83a57c3a416


In [73]:
print('Actual sig : 3045022100f8dac321b0429798df2952d086e763dd5b374d031c7f400d92370ae3c5f57afd0220531207b28b1b137573941c7b3cf5384a3658ef5fc238d26150d8f75b2bcc61e701')

Actual sig : 3045022100f8dac321b0429798df2952d086e763dd5b374d031c7f400d92370ae3c5f57afd0220531207b28b1b137573941c7b3cf5384a3658ef5fc238d26150d8f75b2bcc61e701


In [ ]:
'30450221008ba66acb7ea0cb94d04eb2f0291b141cd705f5faea85d04208cf546b6df8f2a702203df05587ccab303cb44a8f717f5486cb77b52d576b3cf9dbfeedb97a5be80cae' == 

In [75]:
import ecdsa
from ecdsa import SECP256k1
import hashlib
import hmac

def deterministic_sign(private_key_hex, unsigned_transaction_hex):
    # Deserialize the unsigned transaction hex
    unsigned_transaction_bytes = bytes.fromhex(unsigned_transaction_hex)

    # Hash the transaction
    transaction_hash = double_sha256(unsigned_transaction_bytes)

    # Convert private key to bytes
    private_key = bytes.fromhex(private_key_hex)

    # Create an ECDSA private key
    signing_key = ecdsa.SigningKey.from_string(private_key, curve=SECP256k1)

    # RFC 6979 deterministic k generation using HMAC-SHA256
    deterministic_k = ecdsa.rfc6979.generate_k(signing_key.curve.order, signing_key.privkey.secret_multiplier, transaction_hash, hashlib.sha256)

    # Sign the hash using the deterministic k
    signature = signing_key.sign_digest_deterministic(transaction_hash, hashfunc=hashlib.sha256, sigencode=ecdsa.util.sigencode_der, k=deterministic_k)

    # Return the signature as bytes
    return signature

def double_sha256(data):
    return hashlib.sha256(hashlib.sha256(data).digest()).digest()

# Example Usage
private_key_hex = "26F85CE8B2C635AD92F6148E4443FE415F512F3F29F44AB0E2CBDA819295BBD5"
unsigned_transaction_hex = "0200000099197e88ff743aff3e453e3a7b745abd31937ccbd56f96a179266eba786833e682a7d5bb59fc957ff7f737ca0b8be713c705d6173783ad5edb067819bed70be89cb872539fbe1bc0b9c5562195095f3f35e6e13919259956c6263c9bd53b20b7010000001976a914594c2e3da92d1904f7e7c856220f8cae5efb556488ac5424000000000000fffffffff3ae23c3fd63a2e0479888f95c7a8ab221b20add6ac819e9d8953edd1a0cd9240000000001000000"

signature = deterministic_sign(private_key_hex, unsigned_transaction_hex)
print("Deterministic ECDSA Signature:", signature.hex())


TypeError: 'bytes' object is not callable